In [1]:
import pandas as pd
import altair as alt
import numpy as np
#from google.colab import files

df = pd.read_csv('2021natality_clean.csv')

/var/folders/qv/whncf_dn3x79pb78276v5dxw0000gn/T/ipykernel_13333/3061811835.py:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('2021natality_clean.csv')


In [2]:
df.head()

,dob_yy,dob_mm,dob_tt,bfacil,mager,mracehisp,mar_p,dmar,meduc,feduc,...,rf_ghype,rf_inftr,rf_fedrg,rf_artec,dbwt,mm_mtr,mm_plac,mm_rupt,mm_uhyst,mm_aicu
0,2021,1,636,1,22,7,X,1.0,3,4,...,N,N,X,X,3670,N,N,N,N,N
1,2021,1,259,1,31,6,Y,2.0,4,3,...,N,N,X,X,3210,N,N,N,N,N
2,2021,1,223,1,29,1,Y,2.0,4,4,...,N,N,X,X,3997,N,N,N,N,N
3,2021,1,241,1,39,7,Y,2.0,6,5,...,Y,N,X,X,3510,N,N,N,N,N
4,2021,1,503,1,20,5,Y,2.0,3,3,...,N,N,X,X,3100,N,N,N,N,N


In [3]:
subset = df[['dob_yy', 'dob_mm', 'dob_tt', 'no_mmorb', 'mm_mtr', 'mm_plac', 'mm_rupt', 'mm_uhyst', 'mm_aicu']]

# dob_yy = 2021
# DOB_MM = month (1 -12)
# DOB_TT = time 0000 - 2359, or 9999 = unk
# dbwt = birth weight in grams (9999 = unk)

# no_mmorb = No Maternal Morbidity Reported. 
# mm_mtr = maternal transfusion
# mm_plac = perineal laceration
# mm_rupt = ruptured uterus
# mm_uhyst = unplanned hysterectomy
# mm_aicu = admit to icu

#1 = true, 0 = false, 9 = unk

subset

,dob_yy,dob_mm,dob_tt,no_mmorb,mm_mtr,mm_plac,mm_rupt,mm_uhyst,mm_aicu
0,2021,1,636,1,N,N,N,N,N
1,2021,1,259,1,N,N,N,N,N
2,2021,1,223,1,N,N,N,N,N
3,2021,1,241,1,N,N,N,N,N
4,2021,1,503,1,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...
3669923,2021,12,1901,1,N,N,N,N,N
3669924,2021,10,520,9,U,U,U,U,U
3669925,2021,12,242,1,N,N,N,N,N
3669926,2021,6,2146,1,N,N,N,N,N


In [4]:
# focus just on maternal morbididty first 
morb = subset[['dob_mm', 'no_mmorb', 'dob_tt']]
morb

,dob_mm,no_mmorb,dob_tt
0,1,1,636
1,1,1,259
2,1,1,223
3,1,1,241
4,1,1,503
...,...,...,...
3669923,12,1,1901
3669924,10,9,520
3669925,12,1,242
3669926,6,1,2146


In [5]:
# check for missing data
print(morb.isna().sum())

dob_mm      0
no_mmorb    0
dob_tt      0
dtype: int64


In [6]:
deaths = morb['no_mmorb']
print((deaths == 1).sum())
print((deaths == 0).sum())
print((deaths == 9).sum())

3610907
52009
7012


In [18]:
# group and filter out unknown values
morb_group = morb.groupby(['dob_mm', 'no_mmorb']).count()
morb_group = morb_group.reset_index()
morb_group = morb_group.loc[morb_group['no_mmorb'] != 9]
#morb_group

In [8]:
# recode
morb_group['no_mmorb'] = morb_group['no_mmorb'].map({1: 'Survival', 0: 'Death'})
morb_group['dob_mm'] = morb_group['dob_mm'].map({1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May',
                                                6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October',
                                                11: 'November', 12: 'December'})
morb_group

,dob_mm,no_mmorb,dob_tt
0,January,Death,3772
1,January,Survival,273368
3,February,Death,3739
4,February,Survival,262661
6,March,Death,4290
7,March,Survival,298431
9,April,Death,4326
10,April,Survival,288955
12,May,Death,4355
13,May,Survival,296656


In [9]:
# set up a sorted bar plot showing cancer deaths in 2019
plot = alt.Chart(morb_group).mark_bar().encode(
    x = alt.X('dob_mm:O', title = 'Month',  sort= ["January", "February", "March", "April" , 
                                                   "May", "June", "July",  "August", 'September', 
                                                   'October', 'November', 'December']),
    y = alt.Y('dob_tt:Q', title = 'Birth Count'),
    color = alt.Color('no_mmorb:N', title = 'Maternal Morbidity', scale=alt.Scale(
        range=[' #0F52BA', '#87CEEB'])),
    tooltip = [alt.Tooltip('dob_tt:Q', title ='Count')]
).properties(
    title= 'USA Maternal Morbidity in 2021',
    width = 600,
    height = 300
)
#plot

# angle the x-axis labels and include full label
plot2 = plot.configure_axisBottom(
    labelAngle = 45,
    labelLimit = 0
)
plot2

alt.Chart(...)

In [10]:
# filter out the 9 

# plot birth by hours as clock

# normalize deaths per month ot total births in that month

# also have raw count for death/live

In [11]:
# filter unknown times and outcomes
morb = morb.loc[morb['dob_tt'] != 9999]
morb= morb.loc[morb['no_mmorb'] != 9]

In [12]:
# create bins to count the number of births per hour
morb['bins'] = pd.cut(x=morb['dob_tt'], bins=[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 
                                             1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 
                                             2100, 2200, 2300, 2400], labels=['0:00-1:00', '1:01-2:00', '2:01-3:00', '3:01-4:00', '4:01-5:00',
                         '5:01-6:00', '6:01-7:00', '7:01-8:00', '8:01-9:00', '9:01-10:00', '10:01-11:00',
                         '11:01-12:00', '12:01-13:00', '13:01-14:00', '14:01-15:00', '15:01-16:00',
                         '16:01-17:00', '17:01-18:00', '18:01-19:00', '19:01-20:00', '20:01-21:00',
                         '21:01-22:00', '22:01-23:00', '23:01-24:00'])

morb['dob_mm'] = morb['dob_mm'].map({1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May',
                                                6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October',
                                                11: 'November', 12: 'December'})
# recode )
                  
#morb['bins'] = morb_group['bins'].map({'(100, 200]': '0:00 - 1:00', '(100, 200]': '1:01'})

morb_group2 = morb.groupby(['bins', 'dob_mm', 'no_mmorb']).count()
morb_group2 = morb_group2.reset_index()
morb_group2['no_mmorb'] = morb_group2['no_mmorb'].map({1: 'Survival', 0: 'Death'})
morb_group2

,bins,dob_mm,no_mmorb,dob_tt
0,0:00-1:00,April,Death,149
1,0:00-1:00,April,Survival,9146
2,0:00-1:00,August,Death,144
3,0:00-1:00,August,Survival,10509
4,0:00-1:00,December,Death,137
...,...,...,...,...
571,23:01-24:00,November,Survival,9786
572,23:01-24:00,October,Death,156
573,23:01-24:00,October,Survival,10324
574,23:01-24:00,September,Death,173


In [19]:
# create selector for outcome, add to dropdown
outcome = morb_group2['no_mmorb'].unique()

outcome_dropdown = alt.binding_select(options=outcome)

outcome_select = alt.selection_single(fields=['no_mmorb'], bind=outcome_dropdown, 
                                     name= "Maternal Morbidity Outcome", init = {'no_mmorb': outcome[0]})
outcome_select

Selection('Maternal Morbidity Outcome', SelectionDef({
  bind: BindRadioSelect({
    input: 'select',
    options: array(['Death', 'Survival'], dtype=object)
  }),
  fields: ['no_mmorb'],
  init: {'no_mmorb': 'Death'},
  type: 'single'
}))

In [20]:
# creat a selector for month, add to legend
month_selection = alt.selection_single(fields=['dob_mm'], bind='legend')
month_selection

Selection('selector002', SelectionDef({
  bind: 'legend',
  fields: ['dob_mm'],
  type: 'single'
}))

In [24]:
line = alt.Chart(morb_group2).mark_line(point=True).encode(
  x = alt.X('bins', title = 'Time', sort = ['0:00-1:00', '1:01-2:00', '2:01-3:00', '3:01-4:00', '4:01-5:00',
                         '5:01-6:00', '6:01-7:00', '7:01-8:00', '8:01-9:00', '9:01-10:00', '10:01-11:00',
                         '11:01-12:00', '12:01-13:00', '13:01-14:00', '14:01-15:00', '15:01-16:00',
                         '16:01-17:00', '17:01-18:00', '18:01-19:00', '19:01-20:00', '20:01-21:00',
                         '21:01-22:00', '22:01-23:00', '23:01-24:00']),
  y = alt.Y('dob_tt:Q', title = 'Births'),
  color = alt.Color('dob_mm:N', title = 'Month',
                    sort = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August',
                           'September', 'October', 'November', 'December']),
  opacity=alt.condition(month_selection, alt.value(1), alt.value(0.2)), # add opacity
  tooltip = [alt.Tooltip('dob_tt:Q', title ='Count')]
).properties( 
    title = 'Births per hour in 2021',
    width = 600
).add_selection(
    outcome_select
).transform_filter(
    outcome_select
).add_selection(
month_selection
)

# angle the x-axis labels and include full label
line2 = line.configure_axisBottom(
    labelAngle = 45,
    labelLimit = 0
)
line2

alt.Chart(...)